# Continuity of RB 
Code to generate plot which shows continuity of RB

In [ ]:
import numpy as np
import redundancy_bottleneck as rb

# Channels from Y=(X1,X2) to X1 and X2
src_cond_dists = (np.array([[1,1,0,0],[0,0,1,1]]),
                  np.array([[1,0,1,0],[0,1,0,1]]))

eps_vals  = np.logspace(-2,0,20)
pred_vals = []

R         = 1e-2                  # Desired level of compression
beta_vals = np.logspace(-3,2,100) # Set of possible beta

last_sol = None
for eps in eps_vals:
    # probabilities of:      00      01    10       11
    pY       = np.array([.5-eps/4, eps/4, eps/4, .5-eps/4])

    def f(beta):
        dd = rb.get_rb_value(beta=beta, pY=pY, src_cond_dists=src_cond_dists, num_retries=10)
        return dd.compression, dd.prediction

    min_f = f(beta_vals.max())
    if min_f[0] <= R:
        pred = min_f[1]
    else:
        comp_pred_vals = np.array([ f(beta) for beta in beta_vals ])
        
        # Interpolate compression and prediction values to estimate overall RB curve
        coef = np.polyfit(comp_pred_vals[:,0], comp_pred_vals[:,1], deg=1)
        # Calculate interpolated prediction value at compression R
        pred = np.poly1d(coef)(R)
        
    print("eps=%6.4f pred=%7.4f"%(eps, pred))
    pred_vals.append(pred)


In [ ]:
# Plotting stuff
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib


matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['text.usetex'] = True 
matplotlib.rcParams['text.latex.preamble']="""
\\usepackage{newtxtext}
\\usepackage{bm}
"""

sns.set_style('white') 


# Redundancy bottleneck plot
plt.figure(figsize=(4,3))
plt.xlabel(r'$\epsilon$')
br_color = 'k'
plt.plot(eps_vals, pred_vals, label=r'$I_\textrm{RB}(R)$')
plt.ylabel('Redundancy')
# Blackwell redundancy plot
plt.plot(eps_vals, 0*eps_vals, c=br_color, label=r'$I_\cap$')
plt.scatter([0,], [1,], c=br_color, zorder=10)
plt.scatter([0,], [0,], facecolor='white', edgecolor=br_color, zorder=10)
plt.legend(frameon=False, handlelength=1.5)

fname = 'continuity.pdf'
plt.savefig(fname, bbox_inches='tight')

